# 인공지능 작사가 만들기

OpenAI 재단에서 만든 GPT-3는 자동회귀 언어 딥러닝 모델로 인간과 같은 문자를 만들어 낸다. 2,048개의 토큰 길이의 문맥과 1,750억개의 파라미터들을 활용한 기계학습 모델인 트랜스포머의 표준 아키텍쳐이다.

## "generative pretraining"을 통해 학습한 GPT-3
GPT-3는 다음 토큰이 무엇인지 예측하는 것을 의미하는 "generative pretraining"의 방식으로 학습한다.

## 시퀀스 데이터와 친해지자

시퀀스는 영화, 전기분야 등에서 많이 쓰인다. 문장들은 물론 주가, 날짜, 드라마까지 나열된 데이터를 의미하는 시퀀스 데이터가 널리 사용된다. 꼭 앞 뒤의 요소들이 각각 연관성을 가져야 하는 것이 아니지만, 연관성이 있다면 예측을 할 때 자동회귀가 가능하다.

## 문장을 만드는 방법

'나는', '밥을' ,[  ]라는 빈칸을 채운다면, '먹는다'가 들어갈 확률이 가장 높아보인다. 이것이 우리가 문장을 만들어볼 방식이다.
이렇게 확률이라는 건 학습데이터에서 통계방식으로 확률이 높은 것을 출력하도록 하는 것이다. 이런 방식에 가장 유리한 인공지능 모델 중 하나가 RNN 순환신경망이다.

나는, 밥을, 먹었다 세 토큰을 순차적으로 생성한다면, '밥을' 뒤에 '먹었다'가 온다는 것은 통계적일 수 있지만, '나는' 뒤에 '밥을'이 나오는 건 별다른 통계적인 방식은 아니다. 무작위적인 방식이다.

p(먹었다|나는, 밥을, 맛있게)라는 확률값은 매우 높아보이지만, p(밥을|나는)인가는 적게 나올것이다. 그렇다고 '나는' 뒤에 '밥을'이 나오면 틀렸다는 것이 아니다. 하지만 '나는' 뒤에 확률적으로 나올 수 있는 것이 너무 많아서 불확실성이 매우 높아 보인다.

n-1 개의 단어 시퀀스 w_1, ... , w_(n-1)이 주어졌을 때, n번째 단어 w_n으로 무엇이 올지 예측하는 모델을 언어모델, language model이라고 부르며 파라미터 theta로 모델링하는 언어 모델을 다음과 같이 표현할 수 있다.
P(w_n | w1, ... , w_(n-1);theta)

## 언어모델 학습 방법
n-1번째까지 단어 시퀀스가 x_train, n번째 단어가 y_train이 되는 데이터셋으로 학습한다.

이렇게 학습된 언어모델을 학습모드가 아닌 테스트 모드로 가동하면, 일정 단어 시퀀스가 주어진다면 다음 단어를 순차적으로 예측할 것이다. 이게 바로 텍스트 생성이자 작문이다.

언어 모델의 입력 문장 :  <start> 나는 밥을 먹었다
언어 모델의 출력 문장 : 나는 밥을 먹었다 <end>

각각의 단어들이 토큰화 하고, 단어사전을 만들어 데이터를 숫자로 변환시켜준다. 이렇게 백터화된 숫자데이터를 텐서라고 칭한다.

### 텐서란 무엇인가?


1. Tensor는 배열의 집합이다.
2. 차원의 수는 Rank와 같은말이다.
3. 배열의 차원에따라 불리는 이름이 달라진다.
  - 0차원 : 스칼라
  - 1차원 : 백터
  - 2차원 : 행렬
  - 3이상의 n차원 : n-텐서

## 이번에 사용해볼 언어모델

1. Embedding : 단어를 추상적으로 변환
2. RNN(LSTM)1 & RNN(LSTM)2 : 문장을 순차적으로 읽으며 단어 간의 연관성을 분석
3. Linear(dense) : RNN에서 만들어낸 결과를 바탕으로 생성할 단어를 결정

임베딩 단계에서는 우리가 인덱스에 따른 단어사전을 만들어내는 것이다.
embedding_size 는 단어 백터의 차원수 즉 희소 표현의 단위가 된다. 256만 되어도 65,536개의 공간에서 값들을 조합하여 가질 수 있다.

LSTM레이어에서 은닉층 차원수인 hidden_size도 비슷한 역할이다. 

## 프로젝트 실습 시작
본격적으로 멋진 작사가를 만들어보자.

In [1]:
import os, re 
import numpy as np
import tensorflow as tf

import glob
import os

## 데이터 읽어오기

glob 모듈을 사용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 하자.

In [2]:
txt_file_path = os.getenv('HOME')+'/Workspace/project/projects/aiffel/exploration/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088
Examples:
 ['', '', 'All of this and more is for you', 'With love, sincerity and deepest care', 'My life with you I share']


187,088 줄의 데이터를 갖고 있다. 하지만 공백도 있고, 쉼표, 마침표 등 특수기호들도 있다.
정규표현식을 통해서 공백을 기준으로 토큰화하기 전에 전처리를 해주자.

입력된 문장을
1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
2. 특수문자 양쪽에 공백을 넣고
3. 여러개의 공백은 하나의 공백으로 바꿉니다
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
5. 다시 양쪽 공백을 지웁니다
6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿']+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []



# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) <= 15:
        corpus.append(preprocessed_sentence)

        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> all of this and more is for you <end>',
 '<start> with love , sincerity and deepest care <end>',
 '<start> my life with you i share <end>',
 '<start> ever since i met you , baby <end>',
 "<start> i've been wantin' to lay you down <end>",
 "<start> but it's so hard to get you <end>",
 '<start> baby , when you never come around <end>',
 '<start> every day that you keep it away <end>',
 '<start> it only makes me want it more <end>',
 '<start> ooh baby , just say the word <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  23  18 ...   0   0   0]
 [  2  28  33 ...   0   0   0]
 [  2  12 108 ...   0   0   0]
 ...
 [  2  43  43 ...   0   0   0]
 [  2  21  23 ...   0   0   0]
 [  2  43  43 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x10e00e0e0>


In [6]:
print(tensor[:3, :10])

[[    2    23    18    37     8    94    22    26     7     3]
 [    2    28    33     4 10799     8  4951   292     3     0]
 [    2    12   108    28     7     6  1026     3     0     0]]


In [7]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : the
6 : i
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])
#이거 다음에 
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=1337)

print(enc_train.shape)
print(dec_train.shape)

[ 2 23 18 37  8 94 22 26  7  3  0  0  0  0]
[23 18 37  8 94 22 26  7  3  0  0  0  0  0]
(126925, 14)
(126925, 14)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-10-12 16:48:26.266231: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-12 16:48:26.266421: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.66353309e-04, -1.78581628e-04, -5.32389640e-05, ...,
         -1.53557223e-04,  8.23614027e-05,  2.90446478e-04],
        [ 4.93811443e-04, -4.47035214e-04,  2.71178196e-05, ...,
         -3.28287919e-04, -1.08806234e-04,  3.88753368e-04],
        [ 8.00815993e-04, -7.20072945e-04,  2.92140448e-05, ...,
         -2.83185160e-04, -2.02738534e-04,  2.16082262e-04],
        ...,
        [ 5.01510629e-04, -4.67051228e-04, -5.67851122e-04, ...,
          4.41085576e-04, -1.97726578e-04,  6.49013033e-04],
        [ 4.99308109e-04, -6.95969036e-04, -7.10906170e-04, ...,
          2.76658364e-04, -1.50824068e-04,  1.00153324e-03],
        [ 4.84095159e-04, -9.14620003e-04, -8.15102481e-04, ...,
          9.76004303e-05, -8.72062228e-05,  1.29365339e-03]],

       [[ 2.66353309e-04, -1.78581628e-04, -5.32389640e-05, ...,
         -1.53557223e-04,  8.23614027e-05,  2.90446478e-04],
        [ 5.74740116e-04, -1.98117210e-04,  1

In [12]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.


loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
#Loss
# tf.keras.losses.SparseCategoricalCrossentropy : https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy

model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data=(enc_val, dec_val))



Epoch 1/10


2022-10-12 16:48:29.810394: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-12 16:48:31.131222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:48:31.415548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:48:31.638430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:48:33.367439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:48:34.280811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/496 [==============================] - ETA: 0s - loss: 3.4986

2022-10-12 16:50:35.546406: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:50:35.664320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-12 16:50:35.728736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/496 [==============================] - 136s 265ms/step - loss: 3.4986 - val_loss: 3.1651
Epoch 2/10
496/496 [==============================] - 130s 263ms/step - loss: 3.0587 - val_loss: 2.9901
Epoch 3/10
496/496 [==============================] - 130s 261ms/step - loss: 2.8949 - val_loss: 2.8795
Epoch 4/10
496/496 [==============================] - 129s 260ms/step - loss: 2.7742 - val_loss: 2.8035
Epoch 5/10
496/496 [==============================] - 129s 260ms/step - loss: 2.6727 - val_loss: 2.7422
Epoch 6/10
496/496 [==============================] - 128s 259ms/step - loss: 2.5820 - val_loss: 2.6908
Epoch 7/10
496/496 [==============================] - 129s 261ms/step - loss: 2.4993 - val_loss: 2.6486
Epoch 8/10
496/496 [==============================] - 130s 262ms/step - loss: 2.4215 - val_loss: 2.6139
Epoch 9/10
496/496 [==============================] - 130s 262ms/step - loss: 2.3481 - val_loss: 2.5822
Epoch 10/10
496/496 [==============================] - 130s 262ms/step - lo

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [15]:
generate_text(model, tokenizer, init_sentence="<start> we") # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> we can make it all <end> '